[View in Colaboratory](https://colab.research.google.com/github/juanjosr10/naive_bayes/blob/master/naive_bayes_from_scratch.ipynb)

In [0]:
reset -fs

In [2]:
from collections import namedtuple

In [3]:
Data = namedtuple('data', 'id_num label tokens') 

In [5]:
data = Data(id_num=42, label='cat', tokens="🐯 😺 🐩 😺".split())

In [10]:
train = [Data(42,  'cat',  "🐈 🐯 😺 🐩 😺".split()),
         Data(43, 'dog',  "🐶 🐶 🐈 🐶 🐩 🐈 🐶 🐶".split()),
         Data(45, 'cat',  "🐈 🐈 🐶 🐈".split()),
         Data(45, 'cat',  "🐈 🐈 🐈".split()),
         Data(48, 'dog',  "🐶 🐶 🐈 🐩  🐩 🐶 🐶".split()),
        ]



CALCULATE Document Class Priors

In [7]:
labels = {d.label for d in train}

In [8]:
labels


{'cat', 'dog'}

In [9]:
n_docs = len(train)
n_docs

5

In [12]:
from collections import defaultdict

In [13]:
docs_priors = defaultdict(float)

In [17]:
for label in labels:
  docs_priors[label] = sum(1 for d in train if d.label == label) / n_docs
docs_priors

defaultdict(float, {'cat': 0.6, 'dog': 0.4})

Calculate Conditional probabilities of each word for each class


In [19]:
vocab = []
for doc in train:
  vocab.extend(doc.tokens)
vocab

['🐈',
 '🐯',
 '😺',
 '🐩',
 '😺',
 '🐶',
 '🐶',
 '🐈',
 '🐶',
 '🐩',
 '🐈',
 '🐶',
 '🐶',
 '🐈',
 '🐈',
 '🐶',
 '🐈',
 '🐈',
 '🐈',
 '🐈',
 '🐶',
 '🐶',
 '🐈',
 '🐩',
 '🐩',
 '🐶',
 '🐶']

In [20]:
set(vocab)

{'🐈', '🐩', '🐯', '🐶', '😺'}

In [21]:
v = len(set(vocab))
v

5

In [23]:
cond_prob = defaultdict(lambda: defaultdict(float))

In [27]:
for label in labels:
  label_tokens = []
  for doc in train:
    if doc.label == label:
      label_tokens.extend(doc.tokens)
  
  for token in vocab:
    cond_prob[label][token] = (label_tokens.count(token)) / len(label_tokens)
    
cond_prob

defaultdict(<function __main__.<lambda>>,
            {'cat': defaultdict(float,
                         {'🐈': 0.5833333333333334,
                          '🐩': 0.08333333333333333,
                          '🐯': 0.08333333333333333,
                          '🐶': 0.08333333333333333,
                          '😺': 0.16666666666666666}),
             'dog': defaultdict(float,
                         {'🐈': 0.2, '🐩': 0.2, '🐯': 0.0, '🐶': 0.6, '😺': 0.0})})

In [31]:
for label in labels:
  assert round(sum(cond_prob[label].values())) == 1

In [33]:
# predict label for a new document
import operator
from functools import reduce

def product(it):
  return reduce(operator.mul, it, 1)



In [41]:
test = Data(id_num=90, label=None, tokens="😺".split())
# test = Data(id_num=91, label=None, tokens="🐶 🐶".split()) 
# test = Data(id_num=92, label=None, tokens="🐶 😺".split())
# test = Data(id_num=93, label=None, tokens="🐈 🐈 🐶 🐶 🐩 🐯 🐯".split())
# test = Data(id_num=94, label=None, tokens="🐬".split()) # Out of sample prediction

In [42]:
prod_predicted = defaultdict(float)
for label in labels:
  prod_predicted[label] = docs_priors[label] * product(cond_prob[label][t] for t in test.tokens)
  
prod_predicted

defaultdict(float, {'cat': 0.09999999999999999, 'dog': 0.0})